In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from collections import Counter
import keras
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader
from string import punctuation
from string import digits
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
large_classification_data = pd.read_csv("CDs_and_Vintl_sentiment.csv")
large_classification_data.index = range(len(large_classification_data))

## drop unmeanningful reviews
new_large_classification_data=large_classification_data.drop(labels=[1917,43407, 62315,64996,68579,74976,82369,
                                                             90049,98337,101219,139591,130787,267578,
                                                             295804,335877,396767,159574,164137,223911,
                                                             254229,260236,261849,271662,287960,288971,
                                                             374860,378999,380784,383545],axis=0)
new_small_classification_data = new_large_classification_data.sample(10000, replace = False)

In [31]:
## Make Dictionary
reviews = ("\n".join(i for i in new_large_classification_data['text'].astype(str)))
reviews_split = ''.join([c for c in reviews.lower() if c not in punctuation]).split('\n')
all_text = ' '.join(reviews_split)
words = all_text.split()
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

## Embedding the words
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

## Padding
MAX_LEN=100
reviews_ints=np.array(pad_sequences(reviews_ints, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post"))

## Dividing Training and Testing
train_x, val_x, train_y, val_y = train_test_split(reviews_ints, np.array(new_large_classification_data['label']), random_state=2018, test_size=0.1)
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
batch_size = 30
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)

train_on_gpu=torch.cuda.is_available()

In [33]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
    
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [34]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 200
hidden_dim = 128
n_layers = 3


net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

In [35]:
## Training
lr=0.0001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

epochs = 1
counter = 0
print_every = 100
clip= 500 # gradient clipping

if(train_on_gpu):
    net.cuda()

net.train()
for e in range(epochs):
    h = net.init_hidden(batch_size)
    for inputs, labels in train_loader:
        counter += 1
        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
        if counter%20 == 0:
            lr = 0.1
        else:
            lr = 0.005
        if counter>600 & counter%20 != 0:
            lr = 0.0003
        if counter>600 & counter%20 == 0:
            lr = 0.003
        if counter>1200 & counter%20 != 0:
            lr = 0.0001
        if counter>1200 & counter%20 == 0:
            lr = 0.001      
        optimizer = torch.optim.Adam(net.parameters(), lr=lr)      
        h = tuple([each.data for each in h])
        net.zero_grad()
        output, h = net(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        if counter % print_every == 0:
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:
                val_h = tuple([each.data for each in val_h])
                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()     
                try:
                  output, val_h = net(inputs, val_h)
                  val_loss = criterion(output.squeeze(), labels.float())
                  val_losses.append(val_loss.item())
                except:
                  continue
            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/2... Step: 100... Loss: 0.690162... Val Loss: 0.691499
Epoch: 1/2... Step: 200... Loss: 0.699965... Val Loss: 0.689950
Epoch: 1/2... Step: 300... Loss: 0.723616... Val Loss: 0.691549
Epoch: 1/2... Step: 400... Loss: 0.705618... Val Loss: 0.688439
Epoch: 1/2... Step: 500... Loss: 0.693296... Val Loss: 0.700096
Epoch: 1/2... Step: 600... Loss: 0.726101... Val Loss: 0.680286
Epoch: 1/2... Step: 700... Loss: 0.728043... Val Loss: 0.679064
Epoch: 1/2... Step: 800... Loss: 0.662923... Val Loss: 0.681812
Epoch: 1/2... Step: 900... Loss: 0.646635... Val Loss: 0.662257
Epoch: 1/2... Step: 1000... Loss: 0.753550... Val Loss: 0.662815
Epoch: 1/2... Step: 1100... Loss: 0.646821... Val Loss: 0.629696
Epoch: 1/2... Step: 1200... Loss: 0.645959... Val Loss: 0.646430
Epoch: 1/2... Step: 1300... Loss: 0.635687... Val Loss: 0.635857
Epoch: 1/2... Step: 1400... Loss: 0.680378... Val Loss: 0.632134
Epoch: 1/2... Step: 1500... Loss: 0.585046... Val Loss: 0.591564
Epoch: 1/2... Step: 1600... Loss: 

KeyboardInterrupt: ignored

In [36]:
# Get test data loss and accuracy
test_losses = []
num_correct = 0
h = net.init_hidden(batch_size)
net.eval()
for inputs, labels in valid_loader:
    h = tuple([each.data for each in h])
    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    try:    
      output, h = net(inputs, h)
      test_loss = criterion(output.squeeze(), labels.float())
      test_losses.append(test_loss.item())
    except:
      continue
    pred = torch.round(output.squeeze())
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(valid_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.488
Test accuracy: 0.777
